In [1]:
import numpy as np
from __future__ import division

In [2]:
# set some inputs
x = -2; y = 5; z = -4

# perform the forward pass
q = x + y # q becomes 3
f = q * z # f becomes -12

# perform the backward pass (backpropagation) in reverse order:
# first backprop through f = q * z
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4
# now backprop through q = x + y
dfdx = 1.0 * dfdq # dq/dx = 1. And the multiplication here is the chain rule!
dfdy = 1.0 * dfdq # dq/dy = 1

In [3]:
class mul_neural(object):
    def __init__(self,x,y):
        self.x=x
        self.y=y
        
    def forward(self):
        z=self.x * self.y
        return z
    
    def backword(self,dz):
        dx=self.y * dz
        dy=self.x * dz
        return [dx,dy]
    
    

In [4]:
mul=mul_neural(-2,5)
print mul.forward(),'\n',mul.backword(1)

-10 
[5, -2]


In [5]:
#for vectorized operation
# forward pass
W = np.random.randn(5, 10)
X = np.random.randn(10, 3)
D = W.dot(X)

# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) #.T gives the transpose of the matrix
dX = W.T.dot(dD)

In [6]:
print W[1],'\n',X,'\n',D

[-0.61512137 -0.64225289 -0.71275525  0.15587004 -1.2680654   0.16634248
  1.55364416  0.17862038  0.34367691  0.24536899] 
[[ 1.2614398  -0.63082729 -0.53642245]
 [-0.72267091 -1.28969811 -2.28481648]
 [-0.36261013  2.5937057  -0.7167907 ]
 [ 0.24505639  1.24111496 -1.03504687]
 [ 0.68353875  0.94217214 -1.61585296]
 [ 1.10166963  1.8382235  -3.01515155]
 [ 0.64741584 -1.39852177  0.41168859]
 [-1.367836   -0.06596386  1.01532726]
 [ 0.58490113  1.13479943 -0.05362989]
 [-0.93553342  0.81860603  1.30990332]] 
[[ 1.73922315 -4.71168058 -7.87103709]
 [ 0.03432731 -2.92156103  4.81837191]
 [ 0.47872471  0.31887479  2.42742042]
 [ 0.42753835 -9.56168508  0.43822116]
 [-1.16785126 -4.21997996  1.77658109]]


In [7]:
print dD,'\n',dW,'\n',dX

[[-0.55444305 -1.30521158  0.56082814]
 [-0.26263859 -0.6091947   1.11461878]
 [ 0.46600428 -0.62476782 -0.6826852 ]
 [-0.84083875  1.2735857  -2.35660913]
 [-0.8012666   0.80408799 -1.60912995]] 
[[-0.17687426  0.80261939 -3.58628444 -2.33627085 -2.51493312 -4.70106552
   1.69729814  1.41390807 -1.83552488  0.18487658]
 [-0.54491267 -1.57122084 -2.28378473 -1.9741246  -2.55454998 -4.76992152
   1.1408115   1.53113419 -0.90470829  1.20705957]
 [ 1.34816461  2.02880454 -1.30009932  0.04539983  0.83301205  1.42331923
   0.89439624 -1.28935423 -0.3998074  -1.8416529 ]
 [-0.59994203  4.34952801  5.29739863  3.81381426  4.43312493  8.52034232
  -3.29569873 -1.3266106   1.07984147 -1.25773243]
 [-0.65481679  3.21858794  3.52952436  2.46713505  2.81000993  5.44713303
  -2.30574768 -0.59083295  0.53011431 -0.69996171]] 
[[ 1.95284299 -4.22893776  2.28818541]
 [-2.92285099  0.54937627 -2.37428382]
 [ 3.83479554 -1.8716641   2.49345124]
 [ 1.04583229  1.9651884   2.9744204 ]
 [-1.58569548  5.761

In [8]:
def inti_two_layer_model(input_size,hidden_layer_size,output_size):
    #initialize model
    model={}
    model['w1']=0.0001 * np.random.randn(input_size,hidden_layer_size)
    model['w2']=0.0001 * np.random.randn(hidden_layer_size,output_size)
    model['b1']=np.zeros(hidden_layer_size)
    model['b2']=np.zeros(output_size)
    return model



In [9]:
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

In [10]:
x_train = np.transpose(unpickle('cifar-10/data_batch_1')['data'])
x_train = x_train[:,0:20]
y_train = [unpickle('cifar-10/data_batch_1')['labels']]
y_train = np.transpose(y_train)[0:20,:].T
print y_train[0][2],'\n'
print np.shape(x_train),'\n',np.shape(y_train)

9 

(3072, 20) 
(1, 20)


In [11]:
model= inti_two_layer_model(32*32*3, 50, 10)

In [12]:
def two_layer_net(x_train,model,y_train,reg):
    l1=np.dot(x_train,model['w1'])
    l2=np.dot(l1,model['w2'])
    # l2 is 20 x 10 matrix here other wise it is N x 10 matrix
    l2=np.exp(l2)
    su=l2.sum(axis=0)
    loss= l2/su
    t=0
    for i in range(0,20):
        val= -(np.log(loss[i][y_train[i]]))
        t= t + val
    
    return (t/20)+reg
    

In [13]:
x_train=np.transpose(x_train)
y_train=np.transpose(y_train)
print np.shape(x_train),'\n',np.shape(y_train)

(20, 3072) 
(20, 1)


In [14]:
loss=two_layer_net(x_train,model,y_train,0)

In [15]:
print loss

[2.9957191]


In [16]:
net = TwoLayerNet(32*32*3, 50, 10)

NameError: name 'TwoLayerNet' is not defined

In [17]:
stats = net.train(x_train, y_train, x_train, y_train,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.5, verbose=True)

NameError: name 'net' is not defined

In [18]:
from __future__ import division
a=np.array([[1,2,3],[4,5,6]])
b=a/np.sum(a, axis=1, keepdims=True)
print b

[[0.16666667 0.33333333 0.5       ]
 [0.26666667 0.33333333 0.4       ]]


In [19]:
print np.zeros(10)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
class TwoLayerNet(object):
    
    def __init__(self, input_size, hidden_size, output_size, std=0.0001):
        self.params = {}
        self.params['w1'] = std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['w2'] = std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def loss(self, X, y=None, reg=0.0):
        # unpack the variable
        w1, b1 = self.params['w1'], self.params['b1']
        w2, b2 = self.params['w2'], self.params['b2']
        N, D = X.shape
        
        #compute the forward pass
        scores=None
        
        z1=np.dot(X,w1) + b1
        a1 = np.maximum(0, z1) # pass through ReLU activation function
        # we can also pass through sigmoid function
        scores=np.dot(a1,w2) + b2
        
        if y is None:
            return scores
        
        # other wise compute the loss
        
        loss=None
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
        
        #t=0
        
        corect_logprobs = -np.log(probs[range(N), y])
        data_loss = np.sum(corect_logprobs) / N
        reg_loss = 0.5 * reg * np.sum(w1 * w1) + 0.5 * reg * np.sum(w2 * w2)
        loss = data_loss + reg_loss
        
        #for i in range(0,N):
        #    val= -(np.log(probs[i][y[i][0]]))
        #   t= t + val 
        #data_loss = t/N
        #corect_logprobs = -np.log(probs[range(N), y])
        #data_loss = np.sum(corect_logprobs) / N
    
        
        grads={}
        
        dscores=probs
        dscores[range(N),y] -= 1
        dscores /= N
        
        grads['w2'] = np.dot(np.transpose(a1), dscores)
        grads['b2'] = np.sum(dscores, axis=0)
        
        dhidden = np.dot(dscores, w2.T)
        # backprop the ReLU non-linearity
        dhidden[a1 <= 0] = 0
        # finally into W,b
        grads['w1'] = np.dot(X.T, dhidden)
        grads['b1'] = dhidden#np.sum(dhidden, axis=0)
        
        # add regularization gradient contribution
        grads['w2'] += reg * w2
        grads['w1'] += reg * w1
        
        return loss, grads
    
    def train(self, X, y, X_val, y_val, learning_rate=1e-3, learning_rate_decay=1, reg=1e-5, num_epochs=200, verbose=True):
        
        num_train = X.shape[0]
        #iterations_per_epoch = max(num_train / batch_size, 1)
        
        # Use SGD to optimize the parameters in self.model
        loss_history = []
        train_acc_history = []
        val_acc_history = []
        
        for it in xrange(num_epochs):
            
            #
            #X_batch=None
            #y_batch=None
            
            #sample_indices = np.random.choice(np.arange(num_train), batch_size)
            #X_batch = X[sample_indices]
            #y_batch = y[sample_indices]
    
            
            
            # Compute loss and gradients using the current minibatch
            #loss, grads = self.loss(X_batch, y_batch, 0.0)
            loss, grads = self.loss(X, y=y, reg=reg)
            #loss_history.append(loss)
            
            self.params['w1'] =self.params['w1'] - (learning_rate * grads['w1'])
            self.params['b1'] =self.params['b1'] - (learning_rate * grads['b1'])
            self.params['w2'] =self.params['w2'] - (learning_rate * grads['w2'])
            self.params['b2'] =self.params['b2'] - (learning_rate * grads['b2'])
            
            #if verbose and it % 10 == 0:
            #print 'iteration %d / %d: loss %f' % (it, num_epochs, loss)
                
            # Every epoch, check train and val accuracy and decay learning rate.
            #
           
            train_acc = (self.predict(X) == y).mean()
            val_acc = (self.predict(X_val) == y_val).mean()
            
            print 'iteration %d / %d: loss %f : training accuracy %f, and val accuracy %f' % (it, num_epochs, loss,train_acc, val_acc)
            
            #train_acc_history.append(train_acc)
            #val_acc_history.append(val_acc)
            
            
            learning_rate *= learning_rate_decay
             
                
        #return {
           #'loss_history': loss_history,
          # 'train_acc_history': train_acc_history,
         #  'val_acc_history': val_acc_history,
        #}
        
    
    def predict(self, X):
        
        y_pred = None
        
        z1 = np.dot(X,self.params['w1']) + self.params['b1']
        a1 = np.maximum(0, z1) # pass through ReLU activation function
        scores = np.dot(a1,self.params['w2']) + self.params['b2']
        y_pred = np.argmax(scores, axis=1)
    
        return y_pred
    
    


        


In [21]:
x_train = np.transpose(unpickle('cifar-10/data_batch_1')['data'])
x_train=np.transpose(x_train)
#x_train=np.transpose(x_train)[0:20,:]
y_train = [unpickle('cifar-10/data_batch_1')['labels']]
y_train = np.transpose(y_train)
#y_train = np.transpose(y_train)[0:20,:]
print np.shape(x_train),'\n',np.shape(y_train),'\n',y_train[6][0]

(10000, 3072) 
(10000, 1) 
2


In [22]:
net=TwoLayerNet(32*32*3, 50, 10)

In [23]:
loss,g=net.loss(x_train,y_train)
print loss


23025.819902107327


In [12]:
net.train(x_train,y_train,x_train,y_train,learning_rate=1e-1,learning_rate_decay=1,num_epochs=10,reg=0.0)

NameError: name 'net' is not defined